## Дифференциально экспрессированные гены
---
*Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком*

### Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор  генов, которые позволили  бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).
![1](img/w4_task_1.png)
Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей. 

**Секвенирование** — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

### Практическая значимость изменения
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется **fold change** (кратность изменения). Определяется она следующим образом:

$$ \begin{equation*}
F_c(C,T) = 
 \begin{cases}
   \frac{T}{C} &T>C\\
   -\frac{C}{T} &T<C
 \end{cases}
\end{equation*} $$
 

где  $C$, $T$ — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scipy 
import numpy as np
import pandas as pd
from statsmodels.stats.weightstats import *
import statsmodels.stats.multitest as smm

In [3]:
data = pd.read_csv('data/gene_high_throughput_sequencing.csv', header=0)
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [4]:
data.shape == data.dropna().shape # пропусков в данных нет

True

### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

* для групп **normal (control)** и **early neoplasia (treatment)**
* для групп **early neoplasia (control)** и **cancer (treatment)**

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

**1.** Разделим данные групп:

In [5]:
normal = data.loc[data.loc[:, 'Diagnosis'] == 'normal', :]
early_neoplasia = data.loc[data.loc[:, 'Diagnosis'] == 'early neoplasia', :]
cancer = data.loc[data.loc[:, 'Diagnosis'] == 'cancer', :]

**2.** Сформируем отдельный Data Frame с уровнями значимости t-критерия Стьюдента для пар групп для каждого гена и заполним:

In [6]:
gens_list = data.columns[2:]
ttest_p = pd.DataFrame(
    index=['normal vs early neoplasia',
           'early neoplasia vs cancer'],
    columns=gens_list)

In [7]:
%%time
for gen in gens_list:
    ttest_p.loc['normal vs early neoplasia', gen] = scipy.stats.ttest_ind(
        normal[gen], 
        early_neoplasia[gen],
        equal_var = False).pvalue
    ttest_p.loc['early neoplasia vs cancer', gen] = scipy.stats.ttest_ind(
        early_neoplasia[gen],
        cancer[gen],
        equal_var = False).pvalue

Wall time: 16.9 s


In [8]:
ttest_p.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
normal vs early neoplasia,0.690766,3.17853e-05,0.0602727,0.826429,0.0498762,0.144267,0.329108,0.0237124,0.240148,0.0379328,...,0.463274,0.806008,0.424543,0.740591,0.414922,0.640904,0.830134,0.670395,0.793925,0.661031
early neoplasia vs cancer,0.413735,0.653429,0.0795556,0.287581,0.463292,0.00768133,0.481306,0.57883,0.00074024,0.712687,...,0.107366,0.458907,0.893433,0.467608,0.881584,0.659369,0.330617,0.542939,0.565753,0.63901


**3.** Посчитаем количество статистически значимых отличий:

In [9]:
n_stat_1_group = np.sum(ttest_p.loc['normal vs early neoplasia', :] < 0.05)
n_stat_2_group = np.sum(ttest_p.loc['early neoplasia vs cancer', :] < 0.05)
n_stat_all = n_stat_1_group + n_stat_2_group
print(f'Cтатистически значимых отличий средних значений степени активности генов:\n'\
     f'1. В группе normal (control) и early neoplasia (treatment): {n_stat_1_group}\n'\
     f'2. В группе early neoplasia (control) и cancer (treatment): {n_stat_2_group}\n'
     f'3. Всего: {n_stat_all}')

Cтатистически значимых отличий средних значений степени активности генов:
1. В группе normal (control) и early neoplasia (treatment): 1575
2. В группе early neoplasia (control) и cancer (treatment): 3490
3. Всего: 5065


### Часть 2: поправка методом Холма
В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

Обратите внимание, что

* применять поправку на множественную проверку  нужно  ко всем  значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.  
* при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

**1.** Сформируем Data Frame со скоректированными значениями уровня значимости:

In [10]:
ttest_p_corr_holm = ttest_p.copy()

In [11]:
ttest_p_corr_holm.loc['normal vs early neoplasia', :] = smm.multipletests(
    ttest_p.loc['normal vs early neoplasia', :],
    alpha = 0.025,
    method = 'holm')[1]
ttest_p_corr_holm.loc['early neoplasia vs cancer', :] = smm.multipletests(
    ttest_p.loc['early neoplasia vs cancer', :],
    alpha = 0.025,
    method = 'holm')[1]
ttest_p_corr_holm.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
normal vs early neoplasia,1,0.500174,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
early neoplasia vs cancer,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


**2.** Посмотрим сколько статистически значимых отличий осталось после поправки методом Холма:

In [12]:
n_stat_1_group_corr_holm = np.sum(ttest_p_corr_holm.loc['normal vs early neoplasia', :] < 0.025)
n_stat_2_group_corr_holm = np.sum(ttest_p_corr_holm.loc['early neoplasia vs cancer', :] < 0.025)
n_stat_all_corr_holm = n_stat_1_group_corr_holm + n_stat_2_group_corr_holm
print(f'Cтатистически значимых отличий средних значений степени активности генов после поправки методом Холма:\n'\
     f'1. В группе normal (control) и early neoplasia (treatment): {n_stat_1_group_corr_holm}\n'\
     f'2. В группе early neoplasia (control) и cancer (treatment): {n_stat_2_group_corr_holm}\n'
     f'3. Всего: {n_stat_all_corr_holm}')                                         

Cтатистически значимых отличий средних значений степени активности генов после поправки методом Холма:
1. В группе normal (control) и early neoplasia (treatment): 2
2. В группе early neoplasia (control) и cancer (treatment): 79
3. Всего: 81


**3.** Отберём для каждой группы гены отличия средних значений в которых значимо, посчитаем для каждого значимого изменения fold change, а также число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

In [13]:
name_stat_1 = ttest_p_corr_holm.loc['normal vs early neoplasia', 
                                    ttest_p_corr_holm.loc['normal vs early neoplasia', :] < 0.025].index
name_stat_2 = ttest_p_corr_holm.loc['early neoplasia vs cancer', 
                                    ttest_p_corr_holm.loc['early neoplasia vs cancer', :] < 0.025].index
F = lambda c, t: t/c if t>=c else -c/t
f_stat_1 = []
for gen in name_stat_1:
    f_stat_1.append(F(normal[gen].mean(), early_neoplasia[gen].mean()))

f_stat_2 = []
for gen in name_stat_2:
    f_stat_2.append(F(early_neoplasia[gen].mean(), cancer[gen].mean()))

In [14]:
print(f'Число значимых изменений, абсолютное значение fold change которых больше, чем 1.5:\n'\
      f'1. В группе normal (control) и early neoplasia (treatment): {len([x for x in f_stat_1 if abs(x) > 1.5])}\n'\
      f'2. В группе early neoplasia (control) и cancer (treatment): {len([x for x in f_stat_2 if abs(x) > 1.5])}\n')

Число значимых изменений, абсолютное значение fold change которых больше, чем 1.5:
1. В группе normal (control) и early neoplasia (treatment): 2
2. В группе early neoplasia (control) и cancer (treatment): 77



### Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$, когда они есть, и будут чаще отклонять $H_0$, когда отличий нет).

В качестве ответа к этому заданию требуется  ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5. 

**1.** Сформируем Data Frame со скоректированными значениями уровня значимости:

In [15]:
ttest_p_corr_fdr_bh = ttest_p.copy()

In [16]:
ttest_p_corr_fdr_bh.loc['normal vs early neoplasia', :] = smm.multipletests(
    ttest_p.loc['normal vs early neoplasia', :],
    alpha = 0.025,
    method = 'fdr_bh')[1]
ttest_p_corr_fdr_bh.loc['early neoplasia vs cancer', :] = smm.multipletests(
    ttest_p.loc['early neoplasia vs cancer', :],
    alpha = 0.025,
    method = 'fdr_bh')[1]
ttest_p_corr_holm.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
normal vs early neoplasia,1,0.500174,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
early neoplasia vs cancer,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


**2.** Посмотрим сколько статистически значимых отличий осталось после поправки методом Бенджамини-Хохберга:

In [17]:
n_stat_1_group_corr_fdr_bh = np.sum(ttest_p_corr_fdr_bh.loc['normal vs early neoplasia', :] < 0.025)
n_stat_2_group_corr_fdr_bh = np.sum(ttest_p_corr_fdr_bh.loc['early neoplasia vs cancer', :] < 0.025)
n_stat_all_corr_holm_fdr_bh = n_stat_1_group_corr_fdr_bh + n_stat_2_group_corr_fdr_bh
print(f'Cтатистически значимых отличий средних значений степени активности генов после поправки методом Бенджамини-Хохберга:\n'\
     f'1. В группе normal (control) и early neoplasia (treatment): {n_stat_1_group_corr_fdr_bh}\n'\
     f'2. В группе early neoplasia (control) и cancer (treatment): {n_stat_2_group_corr_fdr_bh}\n'
     f'3. Всего: {n_stat_all_corr_holm_fdr_bh}')                                         

Cтатистически значимых отличий средних значений степени активности генов после поправки методом Бенджамини-Хохберга:
1. В группе normal (control) и early neoplasia (treatment): 4
2. В группе early neoplasia (control) и cancer (treatment): 832
3. Всего: 836


**3.** Отберём для каждой группы гены отличия средних значений в которых значимо, посчитаем для каждого значимого изменения fold change, а также число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

In [18]:
name_stat_1 = ttest_p_corr_fdr_bh.loc['normal vs early neoplasia', 
                                    ttest_p_corr_fdr_bh.loc['normal vs early neoplasia', :] < 0.025].index
name_stat_2 = ttest_p_corr_fdr_bh.loc['early neoplasia vs cancer', 
                                    ttest_p_corr_fdr_bh.loc['early neoplasia vs cancer', :] < 0.025].index

f_stat_1 = []
for gen in name_stat_1:
    f_stat_1.append(F(normal[gen].mean(), early_neoplasia[gen].mean()))

f_stat_2 = []
for gen in name_stat_2:
    f_stat_2.append(F(early_neoplasia[gen].mean(), cancer[gen].mean()))

In [19]:
print(f'Число значимых изменений, абсолютное значение fold change которых больше, чем 1.5:\n'\
      f'1. В группе normal (control) и early neoplasia (treatment): {len([x for x in f_stat_1 if abs(x) > 1.5])}\n'\
      f'2. В группе early neoplasia (control) и cancer (treatment): {len([x for x in f_stat_2 if abs(x) > 1.5])}\n')

Число значимых изменений, абсолютное значение fold change которых больше, чем 1.5:
1. В группе normal (control) и early neoplasia (treatment): 4
2. В группе early neoplasia (control) и cancer (treatment): 524

